In [116]:
import numpy as np
import pandas as pd
import azureml.dataprep as dprep

In [117]:
ratings = dprep.read_csv(path='data/ml-latest-small/ratings.csv', 
                         inference_arguments=dprep.InferenceArguments.current_culture())

ratings = ratings.drop_columns(columns=['timestamp']).to_long(columns=['userId', 'movieId'])
ratings.get_profile()

,Type,Min,Max,Count,Missing Count,Error Count,Lower Quartile,Median,Upper Quartile,Standard Deviation,Mean
userId,FieldType.DECIMAL,1.0,610.0,100836.0,0.0,0.0,175.315403,324.696584,476.657212,182.618491,326.127564
movieId,FieldType.DECIMAL,1.0,193609.0,100836.0,0.0,0.0,1195.783037,2999.663578,7983.624875,35530.987199,19435.295718
rating,FieldType.DECIMAL,0.5,5.0,100836.0,0.0,0.0,2.888792,3.634298,4.000000,1.042529,3.501557


In [118]:
ratings.head(5)
type(ratings)

azureml.dataprep.api.dataflow.Dataflow

In [119]:
def score(pred_x, x):
    assert (len(pred_x) == len(x)), "Mismatch encountered in prediction and baseline" 
    ideal = np.log2(np.arange(len(x)) + 2)
    return np.sum(pred_x / ideal) / np.sum(x / ideal)

In [120]:
def ideal_score(dataflow, userId, topn):
    ff = dataflow.filter(expression=dprep.col('userId') == userId)
    ff = ff.drop_columns(columns=['userId', 'movieId'])
    ff = ff.sort_desc(columns='rating')
    return ff.head(topn).values.T[0]

In [121]:
x = ideal_score(ratings, 1, 10)
pred = np.array([4, 4, 4, 4, 5, 5, 5, 5, 5, 5])

In [122]:
score(pred, x)

0.8872423084619552